In [1]:
import pandas as pd
from pandas_profiling import ProfileReport

import phik
from phik.report import plot_correlation_matrix
from phik import report

import matplotlib.pyplot as plt

%matplotlib inline

fn = 'E:\\VL_sdrag\\combined_data_all_reduced_omni.csv'



In [3]:
df = df = pd.read_csv(fn,parse_dates=[0], 
                      usecols=['Datetime','1-M_AE_nT', 
                               'SYM/H_INDEX_nT',
                               '400kmDensity',
                               'DAILY_SUNSPOT_NO_','DAILY_F10.7_','3-H_KP*10_',
                               'irradiance (W/m^2/nm)'])

df['d_diff'] = df['400kmDensity'].diff()
df = df.dropna()
df = df.set_index('Datetime')

In [4]:
df.head()

,SYM/H_INDEX_nT,1-M_AE_nT,400kmDensity,DAILY_SUNSPOT_NO_,DAILY_F10.7_,3-H_KP*10_,irradiance (W/m^2/nm),d_diff
Datetime,,,,,,,,
2004-02-01 00:01:00,-2.0,92.0,1.276178e-12,70.0,94.5,23.0,0.005673,-2.124240e-13
2004-02-01 00:02:00,-2.0,130.0,1.230694e-12,70.0,94.5,23.0,0.005673,-4.548400e-14
2004-02-01 00:03:00,-3.0,85.0,1.220807e-12,70.0,94.5,23.0,0.005673,-9.887000e-15
2004-02-01 00:04:00,-3.0,75.0,1.304111e-12,70.0,94.5,23.0,0.005673,8.330400e-14
2004-02-01 00:05:00,-3.0,98.0,1.465795e-12,70.0,94.5,23.0,0.005673,1.616840e-13


In [9]:
1.230694e-12-1.276178e-12

-4.548400000000001e-14

In [5]:
df.describe()

,SYM/H_INDEX_nT,1-M_AE_nT,400kmDensity,DAILY_SUNSPOT_NO_,DAILY_F10.7_,3-H_KP*10_,irradiance (W/m^2/nm),d_diff
count,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06,4.670940e+06
mean,-1.141293e+01,1.763493e+02,1.484615e-12,4.752017e+01,9.761624e+01,1.813174e+01,5.515092e-03,-1.020210e-19
std,1.914744e+01,2.141171e+02,1.470658e-12,5.038690e+01,5.429083e+01,1.419073e+01,5.460839e-04,1.700765e-13
min,-4.900000e+02,1.000000e+00,7.457587e-20,0.000000e+00,6.510000e+01,0.000000e+00,4.873058e-03,-1.175625e-11
25%,-1.800000e+01,3.900000e+01,4.975017e-13,9.000000e+00,7.150000e+01,7.000000e+00,5.050188e-03,-3.611200e-14
50%,-8.000000e+00,8.800000e+01,9.641593e-13,3.100000e+01,8.520000e+01,1.700000e+01,5.328221e-03,4.781500e-16
75%,-1.000000e+00,2.360000e+02,1.931388e-12,7.300000e+01,1.111000e+02,2.700000e+01,5.854947e-03,3.793913e-14
max,1.510000e+02,4.192000e+03,2.631810e-11,2.810000e+02,9.999000e+02,9.000000e+01,7.349350e-03,1.223247e-11


In [7]:
df.shape

(4670940, 8)

In [8]:
profile = ProfileReport(df.sample(100000))
profile.to_file("output.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\murph\anaconda3\envs\sdrag\lib\site-packages\multimethod\__init__.py:184: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return self[tuple(map(self.get_type, args))](*args, **kwargs)
C:\Users\murph\anaconda3\envs\sdrag\lib\site-packages\multimethod\__init__.py:184: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return self[tuple(map(self.get_type, args))](*args, **kwargs)
C:\Users\murph\anaconda3\envs\sdrag\lib\site-packages\multimethod\__init__.py:184: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return self[tuple(map(self.get_type, a

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Listing available correlation
correlations = profile.description_set["correlations"]
print(correlations.keys())



In [ ]:
# DataFrame where row and column names are the names of the original columns
print(correlations["pearson"])

In [ ]:
rep = report.correlation_report(df.sample(1000000), significance_threshold=3, correlation_threshold=0.5)

In [ ]:
aa = df.sample(1000)


In [ ]:


aa.head()
